# MCMC CALIBRATION TECHNICS IN CONTEXT OF  INFECTIOUS DISEASE MODELING

## Prerequies

In [ ]:
# pip install summerepi2 estival jax==0.4.24 jaxlib==0.4.24 nevergrad jupyter pymc numpyro

In [ ]:
# pip install multiprocess
%pip install --upgrade --force-reinstall multiprocess


In [ ]:
# This is required for pymc parallel evaluation in notebooks

import multiprocess as mp
import platform

if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [ ]:
import Calibrate as cal #Runing the calibration process and gathering results
from calibs_utilities import get_all_priors, get_targets, load_data
from models import model1 #All the models we design for the test

from estival.model import BayesianCompartmentalModel





In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel # type: ignore
from summer2.parameters import Parameter


# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est
import estival

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
import estival.model
from estival.model import BayesianCompartmentalModel

# Importing the pymc package

import pymc as pm
from estival.wrappers import pymc as epm

In [ ]:
esp.GammaPrior?

## Calibration Data
If data are needed to define a target, we just need to import it from our file data which we will design correctly

In [43]:
#An example of data for the calibration 

output_labels = {"index": "time", "value": "number infectious"}

# data = pd.Series(
#     {"active_cases":
#      {
#          60.0: 3000.0,
#          80.0: 8500.0,
#          100.0: 21000.0,
#          120.0: 40000.0,
#          140.0: 44000.0,
#          160.0: 30000.0,
#          180.0: 16000.0,
#          200.0: 7000.0,
#      },
#     },
# )

#Reading observed data from a csv file 
#targets_data = pd.read_csv('./data/target_model1.csv', index_col=0) 

#print(targets_data)
#targets_data = pd.Series(targets_data)

#Reading the data from a YAML file
targets_yml = './data/target_yml.yml'
targets = load_data(targets_yml)
targets_data = targets["active_cases"]
targets_data.plot(kind="scatter", labels=output_labels)


In [44]:


#Names of parameters and their ranges
params = {
    "contact_rate": (0.0,0.5),
    "recovery_rate": (0.0,0.8)

}
params
targets = get_targets(targets_yml)
priors = get_all_priors(params)

In [ ]:
est.TruncatedNormalTarget?

## Model Definition and Configuration

A mechanistic model (ODE-Based) model discribing Infectious Disease transmission.

In [ ]:
from models.models import model1

In [ ]:

model_1 = model1()




In [ ]:
# #model1 = get_sir_model(model_config)
# model1.request_output_for_compartments(
#         "infectious_population_size", "infection"
#     )

In [ ]:
parameters = {
    "contact_rate": 0.3,
    "recovery_rate": 0.1,
    #"active_cases_dispersion": 0.5,
}

model_1.run(parameters)


pd.DataFrame(
    {
        "modelled": model_1.get_outputs_df()["infectious"],
        "observed": targets_data,
    }
).plot(kind="scatter", labels=output_labels)

In [ ]:
#Defining  a Bayesian Compartmental Model
bcm = BayesianCompartmentalModel(model_1, parameters, priors, targets)

In [ ]:
bcm.priors

In [ ]:
initial_parameters = {
"contact_rate": 0.15,
"recovery_rate": 0.1,
#"active_cases_dispersion": (0.5,1),
}
#bcm.run(initial_parameters)
for param, ranges in initial_parameters.items():
    print(param)
    print(ranges)

In [40]:
algo = pm.DEMetropolisZ
with pm.Model() as model:

    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)

    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata_DEM = pm.sample(step=[algo(variables, S = 5,proposal_dist = pm.NormalProposal)], initvals = initial_parameters, draws=4000,tune = 1000 ,cores=4,chains=4)


NameError: name 'bcm' is not defined